# Lecture 6: Risk Measures

The previous lecture covered risk mapping, which should output a loss distribution $L_{t+1}. $ This lecture will be conerned with how to make such a distribution useful by introducing several methods for measuring risk. Such methods are used in financial institutions to perform tasks such as pricing insurance premiums, managing assets and determining how much capital one can afford to deploy on an investment given its risk. 

The main topics of discussion are:

 1. Basics of risk measurement
 2. Axiomization of risk measure theory
 3. Important risk measures
 4. Standard techniques for calculating VaR
 


In [1]:
#Importing standard libraries

import numpy as np
from matplotlib import pyplot as plt 
import scipy as sci
import pandas as pd

## Basics of Risk Measurement
There are several ways to measure risk. In fact, we have already covered a few examples throughout this course. For example, the option _greeks_ are measures of an option price's sensitivities to various factors, which is essentially a set of risk measures. However, the greeks are limited in the sense where they cannot be aggrigated (or be combined into a single measure), which is usually necessary in risk management. Another notable risk measure are the scenario-based risk measures.

For example, consider a set of risk factor changes $\chi = \{\vec{x}_1, \dots, \vec{x}_n\}$ and a set of weights $\vec{w} = \{w_1, \dots, w_n\} \in [0,1]^n$. Given a portfolio with an associated loss operator $l_{[t]}$, the risk is now taken to be the following function: 

$$\Psi_{[\chi,w]} = \max\{w_1l_{[t]}(\vec{x_1}),\dots, w_nl_{[t]}(\vec{x_n})\}$$

In other words, the risk is taken to be the maximum loss adjusted for its weight (probability of occurence). Whilst this approach doesn't tend to be highly looked up upon, it is very practical and tends to be used by exchanges such as the Chicago Mercantile Exchange and the Hong Kong Exchanges and Clearing which both use it to determine the margins for traders (see SPAN). 

The problems with scenario-based measures is that they are not derived from any theory and that it is usually difficult to accurately determine the weights. For this reason, we will stick to the more common approach of defining risk measures based on loss distributions. One can justify this method with the following statements:

 - Losses are the focus of risk management. Certain moments of distributions can provide monetary indicators
 - Loss distributions can be summarized on each _aggrigation level_ (individual investment, portfolio, company)
 - Diversification effects can be modeled
 - Loss distributions can be compared with eachother
 
And the two disadvantages being 1. you need a lot of data and 2. you may have incorrect assumptions, fit the data to the wrong model, use wrong risk mapping, etc. Although one can easily tell that the disadvantages are not in the the method itself but how it is implemented (i.e one must be very careful with implementing loss-distribution). Another important point is that it is generally bad practice to set the loss distribution as a risk measure (think about presenting a distribution to a CEO -- he doesn't care, he wants a number, eg. risk is $100MM). For this reason, one must then think about _what_ they want from a loss distribution. For example, an insurance business will likely be concerned with the expected value and/or median of this distribution.



#### Python Implementation

Given that there isn't much python implementation one can do for this section, a good idea would be to create a function that maps an input loss operator `l_t(x)`, weight vector `w` and risk factor change `chi` to an output risk measure `Phi` that assumes a scenario based model.

In [1]:
def Phi(l_t, w, chi):
    risks = [w[i]*l_t(chi[i]) for i in range(len(w))]
    return max(risks)

## Axiomatic Risk Measure Theory

The following section is all based on Artzner et al. (1999) which is linked in the "Further Reading" section. The goal is to define valid or _coherent_ risk measures using a set of axioms, and to use those set of axioms to judge commonly used risk measures such as VaR and expected shortfall.

Consider a probability triple $(\mathbf{\Omega},\mathcal{F}, \mathbb{P})$ and a time horizon $\Delta$. We then denote the set of all random variables on $(\mathbf{\Omega},\mathcal{F})$ as $L^0(\mathbf{\Omega},\mathcal{F}, \mathbb{P})$. A financial risk $\mathcal{M}$ is then said to be a subset of $L^0$, or $\mathcal{M} \subset L^0(\mathbf{\Omega},\mathcal{F}, \mathbb{P})$. 

$\mathcal{M}$ is also said to be a convex cone, meaning that given two risks (RVs) $X, Y \in \mathcal{M}$ and a scalar $a > 0$

$$X + Y \in \mathcal{M}$$

and 

$$aX \in \mathcal{M}$$


A risk measure $\mathcal{\eta}$